In [1]:
from pvlib import modelchain as mc
from pvlib import pvsystem
import pandas as pd
import numpy as np

import math
import random

In [2]:
w = 39
l = 65
maxCuts = 6
zMin = 30
zMax = 70
tMin = 0
tMax = 90
aMin = 0
aMax = 360
res = 6

HELPER FOR MAKING AN INDIVIDUAL

inputs: l- int - length of the initial solar panel
        w- int - width of the initial solar panel
        maxCuts- int - maximum amount of cuts on the initial solar panel
        res- int - resolution of the cut. i.e. the cut will happen every 'res' inches
    
output- a list of bits where a bit represents an inch along the solar panel. the first
        'l' bits represent vertical positions, and the next 'w' bits represent horizontal positions,
        such that each position happens every 'res' inches.

In [3]:
def makeCutBitString(l, w, maxCuts, res):
    
    #figure out the number of the bit strings
    numBits = int(round(((l-1)+(w-1)/res)))

    #fill the list with 0's
    bitList = [0 for i in range(numBits)]

    #get the number of cuts
    numCuts = random.randint(1, maxCuts)

    #Lets put in the 1's that represent the cuts
    for i in range(numCuts):
        val = random.randint(0,numBits-1)

        #make sure that it doesn't replace a value in bitstring which is already a 1
        while bitList[val] == 1:
            val = random.randint(0,numBits)

        #replace the appropriate 0 with a 1
        bitList[val] = 1
        
    #return final bitstring
    return bitList

cbs = makeCutBitString(l,w,maxCuts, res)
print("Cut Bit String: ", cbs)

Cut Bit String:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


HELPER FOR FINDING FACES

inputs: l- int - length of the initial solar panel
        w- int - width of the initial solar panel
        aBitString- a list of bits - with a length equal to (l+w)/res
        res- int - resolution of the cut. i.e. the cut will happen every 'res' inches
    
output: a list of tuples where each tuple has 3 ints representing the info for each face. 
        The ints are the length, width, and area of the face. The whole list contains all of the faces

In [4]:
def getFaceListBitString(l, w, aBitString, res):
    
    # error check
    if len(aBitString) != int(round(((l-1)+(w-1)/res))):
        return "ERROR: the 'l' and 'w' parameters do not correspond to the given bit string"
    
    # get the total number of bits
    numBits = len(aBitString)

    # make bit strings that represent just the length and the width of the solar panel respectively
    lenBitString = aBitString[0:l-1]
    widBitString = aBitString[l-1:]
    
    # initialize list of faces
    faceList = []

    ## get list of the positions where a cut will happen
    # vertical cuts
    vCuts = [i+1 for i in range(len(lenBitString)) if lenBitString[i] == 1]
    # horizontal cuts
    hCuts = [i+1 for i in range(len(widBitString)) if widBitString[i] == 1]

    ## initialize some variables
    vLow = 0
    vHigh = l
    hLow = 0
    hHigh = w

    # make lists storing at what inches chuts are being made
    vCuts = [0] + vCuts + [l]
    hCuts = [0] + hCuts + [w]

    # initalize some vars
    lengths = []
    widths = []

    # fill 'lengths' with the length of each face
    for i in range(len(vCuts)-1):
        vLow = vCuts[0]
        vHigh = vCuts[1]
        length = vHigh - vLow
        lengths.append(length)

        # update vCuts
        vCuts = vCuts[1:]

    # fill 'widths' with the width of each face
    for i in range(len(hCuts)-1):
        hLow = hCuts[0]
        hHigh = hCuts[1]
        width = hHigh - hLow
        widths.append(width)

        # update vCuts
        hCuts = hCuts[1:]

    ## fill faceList with areas
    #for every length
    for i in range(len(lengths)):
        length = lengths[i]
        # for every width
        for j in range(len(widths)):
            width = widths[j]
            area = length*width
            faceList.append(area) # Only getting Area 

    return faceList

fbs = getFaceListBitString(l, w, cbs, res)
print("Face List:", fbs)

Face List: [2106, 429]


MAKES AN INDIVIDUAL

inputs: l- int - length of the initial solar panel
        w- int - width of the initial solar panel
        maxCuts- int - maximum amount of cuts on the initial solar panel
        zMin - int - min possible value of z
        zMax - int - min possible value of z
        tMin - int - min possible value of t
        tMax - int - min possible value of t
        aMin - int - min possible value of a
        aMax - int - min possible value of a
        res- int - resolution of the cut. i.e. the cut will happen every 'res' inches

output: a list of ints - individual whose first ((l-1)+(w-1))/res elements are bits representing where a cut is made
        the other ((maxCuts/2)+1)*((maxCuts/2)+1) elements (assuming maxCuts is even) represent the z, theta, and alpha 
        values. It looks like: [bit1 bit2 bit2 ... z1, t1, a1, z2, t2, a2...]

In [5]:
def makeIndividual(l, w, maxCuts, zMin, zMax, tMin, tMax, aMin, aMax, res):
    
    # error check to see if maxCuts can be defined
    if (maxCuts % 2) == 0:
        maxFaces = ((maxCuts/2)+1)*((maxCuts/2)+1)
    else:
        return "ERROR: You need to figure out how many maxFaces will result when there is an odd number of maxCuts"
    
    aBitString = makeCutBitString(l, w, maxCuts, res)
    
    # initalize list
    paramList = []
    tList = []
    aList = []
    
    for i in range(int(maxFaces)):
        zVal = random.randint(zMin, zMax)
        tVal = random.randint(tMin, tMax)
        aVal = random.randint(aMin, aMax)
        paramList.append(zVal)
        paramList.append(tVal)
        paramList.append(aVal)
        tList.append(tVal)
        aList.append(aVal)
        
    individual = aBitString + paramList
    
    return tList, aList

tAngle, aAngle = makeIndividual(l, w, maxCuts, zMin, zMax, tMin, tMax, aMin, aMax, res)
print("tAngle:", tAngle)
print("")
print("aAngle:", aAngle)

tAngle: [3, 19, 11, 3, 65, 64, 56, 66, 68, 26, 16, 25, 13, 70, 20, 79]

aAngle: [108, 358, 123, 17, 247, 117, 26, 55, 13, 112, 207, 28, 287, 360, 243, 272]


In [6]:
#DataFrame containg time and dates. I used 3 month intervales with different times each day. Can change!
#We have it caluclating from 11Am-7Pm on August 1st.
df = pd.DataFrame({'date': ['2012-07-01 11:00:00','2012-07-01 12:00:00',
                            '2012-07-01 13:00:00','2012-07-01 14:00:00','2012-07-01 15:00:00','2012-07-01 16:00:00'
                           '2012-07-01 17:00:00','2012-07-01 18:00:00','2012-07-01 19:00:00'],
                  }).set_index('date')

#Creating DatetimeIndex Object
df.index = pd.DatetimeIndex(df.index)

In [7]:
#Athens, Ga Lat/long
latitude = 33.957409
longitude = -83.376801

#Needed paramters for the basic chain model
#These are constant through the whole simulation
cec_mod = pvsystem.retrieve_sam('SandiaMod')
cec_inv = pvsystem.retrieve_sam('CECInverter')
sapm = cec_mod['Advent_Solar_AS160___2006_']
cec = cec_inv['ABB__MICRO_0_3_I_OUTD_US_208_208V__CEC_2014_']


#Our basic chain model.
#Surface_tilt is our theta
#Surface_arimuth is our alpha
model = mc.basic_chain(df.index, latitude, longitude , module_parameters = sapm, inverter_parameters = cec,
               surface_tilt = 0, surface_azimuth = 0)
#Droping the columns we dont need
power = model[0].drop(columns = ['i_sc','i_mp','v_oc','i_x','i_xx'])


a = power['2012-07'].append(pd.DataFrame([power['2012-07'].sum(0,1)]))
print(a)

#v_mp is maximum voltage
#p_mp is maximum power
# 0 row is sum

                           v_mp        p_mp
2012-07-01 11:00:00   24.010992    2.304608
2012-07-01 12:00:00   31.921740   27.867212
2012-07-01 13:00:00   31.627224   61.285601
2012-07-01 14:00:00   30.458670   88.850925
2012-07-01 15:00:00   29.310955  108.658472
2012-07-01 18:00:00   27.904549  127.166697
2012-07-01 18:00:00   27.904549  127.166697
2012-07-01 19:00:00   28.278301  123.101516
0                    231.416980  666.401727


In [8]:
tArea = l*w

maxVolt = []
maxWatt = []

for i,j in zip(tAngle,aAngle):
    model = mc.basic_chain(df.index, latitude, longitude , module_parameters = sapm, inverter_parameters = cec,
               surface_tilt = i, surface_azimuth = j)
    #Droping the columns we dont need
    power = model[0].drop(columns = ['i_sc','i_mp','v_oc','i_x','i_xx'])

    a = power['2012-07'].append(pd.DataFrame([power['2012-07'].sum(0,1)]))
    
    maxVolt.append(a.iloc[-1,0])
    maxWatt.append(a.iloc[-1,-1])

print("Voltage of each sub-solar plate at tAngle & aAngle, before scaling")
print("")
print(maxVolt) 

print("")

print("Power of each sub-solar plate at tAngle & aAngle, before scaling")
print("")
print(maxWatt)

Voltage of each sub-solar plate at tAngle & aAngle, before scaling

[231.71409083285107, 235.83800651584824, 232.0361991744375, 232.08765798679923, 233.48787148251037, 242.38346876171298, 247.018365222557, 244.88066655386612, 250.69587939836345, 234.09879232482328, 229.822098236713, 237.6630557782781, 231.1084134651378, 250.58352178432702, 229.87171652959262, 237.29723309519747]

Power of each sub-solar plate at tAngle & aAngle, before scaling

[679.9315286068467, 637.0984204408988, 703.0375456004318, 670.3454886677931, 338.0284591865982, 583.4547461961256, 508.7832679002886, 512.9038536971209, 343.2921347381803, 727.338530242156, 615.0782327042318, 672.9601635951202, 592.9542760683986, 256.38539947508445, 558.9216137786284, 241.19470730644738]


In [9]:
voltageFitness = 0
powerFitness = 0

for i,j,z in zip(fbs, maxVolt, maxWatt): # fbs is Face List / face bit string
    scalingFactor = i / tArea
    voltageFitness = voltageFitness + (j*scalingFactor)
    powerFitness = powerFitness + (z*scalingFactor)

print("Fitness of entire Tree after scaling")
print("Fitness of Individual as Volt: ", voltageFitness)
print("")
print("Fitness of Individual as Watt: ", powerFitness)

Fitness of entire Tree after scaling
Fitness of Individual as Volt:  232.41198425612754

Fitness of Individual as Watt:  672.6828487633786
